In [0]:
#Importing all the requisite Libraries 
import csv
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize
nltk.download('wordnet')
import spacy
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.layers import Dense,Dropout,Embedding,LSTM,Flatten,GRU,SpatialDropout1D,Bidirectional
import gensim
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import classification_report
encoder = LabelEncoder()
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
df = pd.read_csv('/content/drive/My Drive/Dataset/IMDB Dataset.csv')

In [0]:
#Doing the preprocessing.
df.isnull().sum() #checking if we have any NULL values
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [0]:
'''
creating a dictionary for abbreviated terms which will be used to pre process the data
'''

s_dict = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [0]:
'''
The function preprocess() actually helps us to do all the pre processing steps one by one and returns a processed text
'''
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
  text = text.lower()
  text = " ".join([s_dict[word] if word in s_dict else word for word in text.split()])
  text = " ".join([lemma.lemmatize(word) for word in word_tokenize(text)])
  text = re.sub(r'\W',' ',text)
  text = re.sub(r'\s+',' ',text)
  text = " ".join([i for i in text.split() if i not in STOPWORDS])
  return text


In [0]:

'''
Calling the necessary functions for pre processing
'''
df['processed text'] = df['review'].apply(lambda x:preprocess(x))
df['tokenized_sents'] = df['processed text'].apply(lambda row:word_tokenize(row))
df['sentiment']=encoder.fit_transform(df['sentiment'])


In [0]:
df.head(5)

,review,sentiment,processed text,tokenized_sents
0,One of the other reviewers has mentioned that ...,1,one reviewer ha mentioned watching 1 oz episod...,"[one, reviewer, ha, mentioned, watching, 1, oz..."
1,A wonderful little production. <br /><br />The...,1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,I thought this was a wonderful way to spend ti...,1,thought wa wonderful way spend time hot summer...,"[thought, wa, wonderful, way, spend, time, hot..."
3,Basically there's a family where a little boy ...,0,basically family little boy jake think zombie ...,"[basically, family, little, boy, jake, think, ..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter mattei love time money visually stunnin...,"[petter, mattei, love, time, money, visually, ..."


In [0]:
'''
We are calling word2vec model to train the embedding layer
'''
review_lines = df['tokenized_sents'].values
embbed_model = gensim.models.Word2Vec(sentences = review_lines,size = 100,window=5,workers=4,min_count=1)


In [0]:
embbed_model.wv.most_similar('nice')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('good', 0.6844138503074646),
 ('neat', 0.676488995552063),
 ('cool', 0.6455390453338623),
 ('interesting', 0.6008448600769043),
 ('great', 0.5883601903915405),
 ('odd', 0.5803523063659668),
 ('refreshing', 0.57901930809021),
 ('pleasant', 0.5698761343955994),
 ('wonderful', 0.5456319451332092),
 ('cute', 0.5330506563186646)]

In [0]:
filename = '/content/drive/My Drive/word2vec_imdb.txt'
embbed_model.wv.save_word2vec_format(filename,binary = False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
'''
We are again creating the word dictionary of our dataset and then creating
sequence of numbers for all the sentences and then we are doing the padding 
to make all the input data uniform.
'''

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(df['processed text'].values)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(df['processed text'].values)
Padded_X = pad_sequences(X,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
Y = df['sentiment'].values

In [0]:
'''
We are creating our embedding matrix with the use of 
pretrained coeficients of word2vec model
'''

embeding_index = {}
import os
import numpy as np
file = open(os.path.join('','/content/drive/My Drive/word2vec_imdb.txt'),encoding='utf-8')
for line in file :
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeding_index[word] = coefs

num_words = len(word_index)+1
embedding_matrix_w2v = np.zeros((num_words,100))

for word,i in word_index.items():
  if i > num_words:
    continue
  emd_v = embeding_index.get(word)
  if emd_v is not None:
    embedding_matrix_w2v[i] =  emd_v


In [0]:
embedding_matrix_w2v.shape


(95081, 100)

In [0]:
print('shape of Review tensor:',Padded_X.shape)
print('Shape of sentiment shape:',Y.shape)

shape of Review tensor: (50000, 100)
Shape of sentiment shape: (50000,)


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(Padded_X,Y, test_size = 0.3, random_state = 42)

In [0]:
print('Shape of XTrain{},XTest{},YTrain{},Ytest{}:'.format(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape))

Shape of XTrain(35000, 100),XTest(15000, 100),YTrain(35000,),Ytest(15000,):


In [0]:
'''
our RNN model
'''

model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM,weights=[embedding_matrix_w2v], input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.25))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          9508100   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 9,669,101
Trainable params: 9,669,101
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
'''
Here we are training our RNN model and stopping the training once 
our training accuracy reaches 96% to avoid overfitting
'''
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.96):
      print("\nReached 96% accuracy so cancelling training!!!.Stopping the training to avoid overfitting")
      self.model.stop_training = True
callbacks = myCallback()
epochs = 10
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs,
                    validation_split=0.1,verbose=1,callbacks = [callbacks])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 31500 samples, validate on 3500 samples
Epoch 1/10
31500/31500 [==============================] - 499s 16ms/step - loss: 0.3855 - accuracy: 0.8272 - val_loss: 0.2841 - val_accuracy: 0.8883
Epoch 2/10
31500/31500 [==============================] - 493s 16ms/step - loss: 0.2641 - accuracy: 0.8946 - val_loss: 0.2633 - val_accuracy: 0.8897
Epoch 3/10
31500/31500 [==============================] - 490s 16ms/step - loss: 0.1770 - accuracy: 0.9321 - val_loss: 0.2809 - val_accuracy: 0.8951
Epoch 4/10
31500/31500 [==============================] - 489s 16ms/step - loss: 0.1000 - accuracy: 0.9643 - val_loss: 0.3154 - val_accuracy: 0.8900

Reached 96% accuracy so cancelling training!!!.Stopping the training to avoid overfitting


In [0]:
print('Evaluating the model with Test data')
Model_test = model.evaluate(X_test,Y_test,batch_size =128 )
print('test loss, test acc:', Model_test)

Evaluating the model with Test data
15000/15000 [==============================] - 4s 296us/step
test loss, test acc: [0.3204834939400355, 0.8875333070755005]


In [0]:
predicted = model.predict_classes(X_test)

In [0]:
print(classification_report(Y_test, predicted))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      7411
           1       0.90      0.88      0.89      7589

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



In [0]:
'''
Creation of basic LR model with count vectorizer.
'''

from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer 
cv = CountVectorizer(ngram_range=(1,2), binary =True, min_df = 10000)
X = cv.fit_transform(df['processed text'].values).toarray()
Y = df['sentiment'].values
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print('Shape of XTrain{},XTest{},YTrain{},Ytest{}:'.format(X_train2.shape,X_test2.shape,Y_train2.shape,Y_test2.shape))

Shape of XTrain(35000, 44),XTest(15000, 44),YTrain(35000,),Ytest(15000,):


In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train2,Y_train2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred = lr.predict(X_test2)
from sklearn.metrics import classification_report

In [0]:
print(classification_report(Y_test2, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68      7411
           1       0.69      0.71      0.70      7589

    accuracy                           0.69     15000
   macro avg       0.69      0.69      0.69     15000
weighted avg       0.69      0.69      0.69     15000



In [0]:
'''
predict_review() and predict_review_RNN() help us to get a prediction value 
when we pass a review statement to these functions
'''
def predict_review(review):
    rvw = preprocess(review)
    x_rvw= cv.transform([rvw])
    review_predict = lr.predict(x_rvw)
    if review_predict == 0:
      print('Bag of Words Model says User Marked it [Negative]')
    else:
      print('Bag of Words Model says User Marked it [Positive]')

def predict_review_RNN(review):
    rvw = preprocess(review)
    x_rvw= tokenizer.texts_to_sequences([rvw])
    fd = pad_sequences(x_rvw,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
    review_predict = model.predict_classes(fd)
    if review_predict == 0:
      print('Deep RNN model says User Marked it [negative]')
    else:
      print('Deep RNN model says User Marked it [Positive]')
 

In [0]:
predict_review("How can I forget the movie. This was really wonderful and cunning.I never want to forget such a beautiful movie")
predict_review_RNN("How can I forget the movie. This was really wonderful and cunning.I never want to forget such a beautiful movie")

Bag of Words Model says User Marked it [Negative]
Deep RNN model says User Marked it [Positive]


In [0]:
predict_review("Despite the wonderfully packaging, the product was poor")
predict_review_RNN("Despite the wonderfully packaging, the product was poor")

Bag of Words Model says User Marked it [Positive]
Deep RNN model says User Marked it [negative]


In [0]:
predict_review_RNN("The excellent acting and soundtrack could not save this movie.")


Deep RNN model says User Marked it [negative]


In [0]:
predict_review_RNN("There are some outstanding and great performance but still the movie is streched to boredom")

Deep RNN model says User Marked it [Positive]
